In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from  partie1 import GRUEncoder, GRUDecoder
from partie1 import LSTMEncoder, LSTMDecoder
from partie1 import TensorProductEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
##### entiers 0-10 
def generate_data(batch_size, seq_len, min_val=0, max_val=10):
    return torch.randint(min_val, max_val, (batch_size, seq_len))

def precision(predicted, target):
    correct = (predicted.argmax(dim=-1) == target).float()
    return correct.mean().item() * 100

# **GRU**

In [ ]:

input_size = 10
emb_size = 16
hidden_size = 32
output_size = 10

batch_size = 16
seq_len = 10
epochs = 10000
learning_rate = 0.001

encoder = GRUEncoder(input_size, emb_size, hidden_size)
decoder = GRUDecoder(emb_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss mieux adapté pour des classes discrètes
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=learning_rate)

for epoch in range(epochs):
    x = generate_data(batch_size, seq_len)
    target = x.clone()

    hidden = encoder(x)
    reconstructed_x = decoder(hidden, seq_len)

    loss = criterion(reconstructed_x.view(-1, output_size), target.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    accuracy = precision(reconstructed_x, target)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

print("Entraînement terminé")
torch.save(encoder.state_dict(), "./models/gru_enc.pth")
torch.save(decoder.state_dict(), "./models/gru_dec.pth")
print("Poids des modèles sauvegardés.")



# **LSTM**

In [ ]:
input_size = 10
emb_size = 16
hidden_size = 32
output_size = 10

batch_size = 16
seq_len = 10
epochs = 10000
learning_rate = 0.001

encoder = LSTMEncoder(input_size, emb_size, hidden_size)
decoder = LSTMDecoder(emb_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss mieux adapté pour des classes discrètes
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=learning_rate)

for epoch in range(epochs):
    x = generate_data(batch_size, seq_len)
    target = x.clone()

    hidden = encoder(x)
    reconstructed_x = decoder(hidden, seq_len)

    loss = criterion(reconstructed_x.view(-1, output_size), target.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    accuracy = precision(reconstructed_x, target)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

print("Entraînement terminé")
torch.save(encoder.state_dict(), "./models/lstm_enc.pth")
torch.save(decoder.state_dict(), "./models/lstm_dec.pth")
print("Poids des modèles sauvegardés.")

# **TPDN**


## GRU

In [ ]:
from partie1 import GRUEncoder, TensorProductEncoder



input_size = 10
emb_size = 16
hidden_size = 32
output_size = 10

batch_size = 16
seq_len = 10
epochs = 10000
learning_rate = 0.001

encoder = GRUEncoder(input_size, emb_size, hidden_size)
encoder.load_state_dict(torch.load("./models/gru_enc.pth"))

tpdnencoder = TensorProductEncoder(n_roles=seq_len, n_fillers=seq_len, filler_dim=hidden_size, role_dim=hidden_size, hidden_size=hidden_size)

def generate_data(batch_size, seq_len, input_size):
    data = torch.randint(0, input_size, (batch_size, seq_len))  # fillers
    roles = torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1)  # rôles
    return data, roles



criterion = nn.MSELoss()
optimizer = optim.Adam(list(encoder.parameters()) + list(tprencoder.parameters()), lr=learning_rate)


for epoch in range(epochs):
    encoder.eval()
    tpdnencoder.train()

    data, roles = generate_data(batch_size, seq_len, input_size)

    encoder_output = encoder(data).squeeze(0)  

    tpdn_output = tpdnencoder(data, roles) 


    loss = criterion(encoder_output, tpdn_output)

    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

torch.save(tpdnencoder.state_dict(), "./models/tpdn_gru.pth")

## LSTM

In [3]:
input_size = 10
emb_size = 16
hidden_size = 32
output_size = 10

batch_size = 16
seq_len = 10
epochs = 10000
learning_rate = 0.001

encoder = LSTMEncoder(input_size, emb_size, hidden_size)
encoder.load_state_dict(torch.load("./models/lstm_enc.pth"))

tpdnencoder = TensorProductEncoder(n_roles=seq_len, n_fillers=seq_len, filler_dim=hidden_size, role_dim=hidden_size, hidden_size=hidden_size)

def generate_data(batch_size, seq_len, input_size):
    data = torch.randint(0, input_size, (batch_size, seq_len))  # fillers
    roles = torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1)  # rôles
    return data, roles



criterion = nn.MSELoss()
optimizer = optim.Adam(list(encoder.parameters()) + list(tpdnencoder.parameters()), lr=learning_rate)


for epoch in range(epochs):
    encoder.eval()
    tpdnencoder.train()

    data, roles = generate_data(batch_size, seq_len, input_size)

    encoder_output = encoder(data)[0].squeeze(0) # [0] pour avoir uniqument le hidden state et non le cell state

    tpdn_output = tpdnencoder(data, roles) 


    loss = criterion(encoder_output, tpdn_output)

    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

torch.save(tpdnencoder.state_dict(), "./models/tpdn_lstm.pth")

Epoch 0/10000, Loss: 4.1172
Epoch 100/10000, Loss: 0.0143
Epoch 200/10000, Loss: 0.0038
Epoch 300/10000, Loss: 0.0055
Epoch 400/10000, Loss: 0.0277
Epoch 500/10000, Loss: 0.0606
Epoch 600/10000, Loss: 0.1773
Epoch 700/10000, Loss: 0.0931
Epoch 800/10000, Loss: 0.0628
Epoch 900/10000, Loss: 0.0303
Epoch 1000/10000, Loss: 0.0295
Epoch 1100/10000, Loss: 0.0328
Epoch 1200/10000, Loss: 0.0136
Epoch 1300/10000, Loss: 0.0083
Epoch 1400/10000, Loss: 0.0437
Epoch 1500/10000, Loss: 0.0679
Epoch 1600/10000, Loss: 0.0321
Epoch 1700/10000, Loss: 0.0264
Epoch 1800/10000, Loss: 0.0177
Epoch 1900/10000, Loss: 0.0201
Epoch 2000/10000, Loss: 0.0084
Epoch 2100/10000, Loss: 0.0108
Epoch 2200/10000, Loss: 0.0271
Epoch 2300/10000, Loss: 0.0183
Epoch 2400/10000, Loss: 0.0143
Epoch 2500/10000, Loss: 0.0184
Epoch 2600/10000, Loss: 0.0177
Epoch 2700/10000, Loss: 0.0257
Epoch 2800/10000, Loss: 0.0232
Epoch 2900/10000, Loss: 0.0081
Epoch 3000/10000, Loss: 0.0061
Epoch 3100/10000, Loss: 0.0067
Epoch 3200/10000, Lo